# Learning objectives

1. Discuss GTEx RNA-seq
2. Practice data frame manipulation
3. Cluster samples by patterns of gene expression
4. Assess what these clusters represent

# Load packages 

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# GTEx RPKMs

## Load data

In [7]:
# download the data from the GTEx portal
# wget https://storage.googleapis.com/gtex_analysis_pilot_v3/rna_seq_data/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz

# then glance at the file
# gzcat ~/Downloads/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz | less -S

df_rpkms = pd.read_csv("~/Downloads/GTEx_Analysis_RNA-seq_RNA-SeQCv1.1.8_gene_rpkm__Pilot_V3_patch1.gct.gz", skiprows = 2, compression = "gzip", sep = "\t")
df_rpkms

,Name,Description,GTEX-N7MS-0007-SM-2D7W1,GTEX-N7MS-0011-R10A-SM-2HMJK,GTEX-N7MS-0011-R11A-SM-2HMJS,GTEX-N7MS-0011-R1a-SM-2HMJG,GTEX-N7MS-0011-R2a-SM-2HML6,GTEX-N7MS-0011-R3a-SM-33HC6,GTEX-N7MS-0011-R4a-SM-2HMKW,GTEX-N7MS-0011-R5a-SM-2HMK8,...,GTEX-X4LF-0526-SM-3NMB6,GTEX-X4LF-1726-SM-3NMBZ,GTEX-X4XX-0005-SM-3NMCS,GTEX-X4XX-0011-R1B-SM-3P622,GTEX-X4XX-0011-R2A-SM-3P623,GTEX-X4XX-0126-SM-3NMC2,GTEX-X4XX-0626-SM-3NMC1,GTEX-X4XX-1126-SM-3NMBY,GTEX-X4XX-2926-SM-3NMB1,GTEX-X4XX-3026-SM-3NMB2
0,ENSG00000223972.4,DDX11L1,0.000000,0.000000,0.000000,0.000000,0.000000,0.029181,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050261,0.000000
1,ENSG00000227232.3,WASH7P,2.917592,1.958602,5.841671,1.728239,2.315600,3.742634,2.269886,2.442356,...,9.417033,7.007399,6.276984,3.710413,5.769073,12.540883,4.696292,4.555761,9.459983,5.700174
2,ENSG00000243485.1,MIR1302-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.102921,0.000000,0.000000,0.000000,0.000000,0.000000
3,ENSG00000237613.2,FAM138A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ENSG00000240361.1,OR4G11P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015078,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52571,ENSG00000198786.2,MT-ND5,627.419189,3967.486572,1014.433228,4683.750000,2813.970947,2435.256104,3997.514893,5427.200684,...,3997.743164,1589.340210,277.970520,7192.353516,1723.450684,2418.593018,3559.731689,2795.689209,1894.077515,3302.491699
52572,ENSG00000198695.2,MT-ND6,553.020569,2737.471680,787.444824,3694.725342,1631.243652,2049.827148,2676.096191,3723.490234,...,6915.279297,1867.175537,400.203766,8612.575195,1159.216675,4115.834473,5160.529297,2533.044434,1654.201904,3118.078125
52573,ENSG00000210194.1,J01415.21,0.434417,0.000000,0.243951,0.847667,0.000000,0.357784,1.051482,0.811884,...,2.372880,0.214025,0.000000,0.974938,0.199876,0.621196,1.584828,0.217481,0.410828,0.000000
52574,ENSG00000198727.2,MT-CYB,2022.417969,20763.693359,7525.217285,24014.476562,13874.849609,12816.257812,18059.113281,26712.630859,...,7216.080078,9511.128906,1375.219727,24179.337891,3829.971191,10955.970703,10365.626953,20897.306641,7734.621094,12888.306641


In [4]:
df_rpkms.describe()

NameError: name 'df_rpkms' is not defined

In [ ]:
df_rpkms.median(axis = 1)

In [ ]:
roi = df_rpkms.median(axis = 1) > 0
roi

In [ ]:
# subset to genes with median expression > 0 across samples
df_rpkms = df_rpkms.loc[roi,]
df_rpkms.shape

In [ ]:
# download the metadata
# wget https://storage.googleapis.com/gtex_analysis_pilot_v3/annotations/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt

df_metadata = pd.read_csv("~/Downloads/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt")
df_metadata

In [ ]:
# read the help
pd.read_csv?

In [ ]:
df_metadata = pd.read_csv( "/Users/rajivmccoy/Downloads/GTEx_Analysis_Annotations_Sample_DS__Pilot_V3.txt", sep = "\t")
df_metadata

In [ ]:
df_rpkms.columns.values

In [ ]:
sample_names = df_rpkms.columns.values[2:]
sample_names

In [ ]:
df_metadata['SAMPID']

In [ ]:
roi = df_metadata['SAMPID'].isin(sample_names)
df_metadata = df_metadata.loc[roi]
df_metadata

In [ ]:
df_metadata['SAMPID'] == sample_names

In [ ]:
(df_metadata['SAMPID'] == sample_names).value_counts()

## PCA

In [ ]:
pca_input = df_rpkms.iloc[:,2:].to_numpy()
type(pca_input)

In [ ]:
pca_input.shape

In [ ]:
pca_input = pca_input.T
pca_input.shape

In [ ]:
pca_input.mean(axis = 0)

In [ ]:
pca_input.var(axis = 0)

In [ ]:
# standardize the genes to zero mean and unit variance
pca_input_standardized = StandardScaler().fit_transform(pca_input)
pca_input_standardized.mean(axis = 0)

In [ ]:
pca_input_standardized.var(axis = 0)

In [ ]:
pca = PCA(n_components = 10)
pca_output = pca.fit_transform(pca_input_standardized)

In [ ]:
pca_output_df = pd.DataFrame(data = pca_output, 
                             columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])
pca_output_df

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(1, 11, 1), pca.explained_variance_ratio_)

## Plot the data

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = pca_output_df['PC1'], y = pca_output_df['PC2'])

In [ ]:
help(plt.scatter)

In [ ]:
pca_output_df['SMTS'] = df_metadata['SMTS']
pca_output_df

In [ ]:
pca_output_df['Tissue'] = df_metadata['SMTS'].tolist()
pca_output_df

In [ ]:
fig, ax = plt.subplots()
groups = pca_output_df.groupby("Tissue")
for name, group in groups:
    ax.scatter(x = group['PC1'], y = group['PC2'], label = name)

plt.legend()
fig.subplots_adjust(bottom=0.2)

In [ ]:
fig, ax = plt.subplots()
groups = pca_output_df.groupby("Tissue")
for name, group in groups:
    ax.scatter(x = group['PC1'], y = group['PC2'], label = name)

plt.legend(bbox_to_anchor = (1.7, 1), loc = 'upper right', ncol = 2)

In [ ]:
fig, ax = plt.subplots()
groups = pca_output_df.groupby("Tissue")
for name, group in groups:
    ax.scatter(x = group['PC1'], y = group['PC6'], label = name)

plt.legend(bbox_to_anchor = (1.7, 1), loc = 'upper right', ncol = 2)

## Exercise

In [ ]:
# Extract data from a single tissue
# perform PCA
# explore the metadata frame
# is there any variable that is correlated with the top principal components?